<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/009_Multi_Turn_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧠 Memory, Logic & LLM-Powered Event Planner

This notebook focused on **building a multi-turn AI assistant** that can:

- Collect user input across multiple steps
- Use both rules and LLMs to extract structured data
- Adapt mid-conversation based on user feedback
- Store, recall, and update user memory intelligently

---

## 🎯 Core Agent Development Lessons

### 1. **Hybrid Reasoning: LLM + Traditional Code**
You learned how to blend:
- **Rule-based logic** (e.g., `if "birthday" in user_input`)
- **Fallback to LLMs** for fuzzy or unclear inputs

> 🔑 *Lesson:* Agents should use LLMs where helpful, but **fall back on traditional code** for reliability and control.

---

### 2. **Memory Matters**
You designed an agent that:
- Tracks missing info
- Remembers user inputs
- Can summarize the full state

> 🔑 *Lesson:* **Memory is essential** to simulate human-like, goal-directed conversations and prevent repetition or confusion.

---

### 3. **Clarification & Correction Handling**
You added support for:
- Ambiguous user phrases like "this summer"
- Corrections like "Actually, it's a party not a wedding"
- Mid-conversation updates

> 🔑 *Lesson:* Real users change their minds. Agents must handle uncertainty and corrections gracefully.

---

### 4. **Prompt Engineering & Fallback Logic**
You refined prompts like:
```text
"Return ONLY a valid JSON object with keys: intent and destination."
```
You also structured fallback JSON parsing carefully.

> 🔑 *Lesson:* **Good prompts + post-processing** are critical to making LLMs act reliably in a structured workflow.

---

### 5. **Final Output & User Experience**
You created a final confirmation summary from memory, and printed a clean final result — useful for handing off to other systems.

> 🔑 *Lesson:* Agents should **produce useful, structured outputs**, not just chat replies. Clean handoff is key.

---

## 🛠️ Skills Gained

✅ Prompt design  
✅ Multi-turn state tracking  
✅ LLM fallback design  
✅ Rule-LLM hybrid logic  
✅ Error handling & user updates  
✅ Final JSON output generation

---

## 🧭 Why This Matters for AI Agent Dev

This project mirrors what *real-world AI agents* must do:

| Real Agent Need                  | You Practiced It |
|----------------------------------|------------------|
| Structured task completion       | ✅ Event fields |
| Handling vague user inputs       | ✅ “this summer” |
| Memory across turns              | ✅ Memory dict |
| Logic branching based on state   | ✅ Question flow |
| LLM post-processing              | ✅ JSON extraction |






# 🧠 Memory-Based Event Planning Agent

In [4]:
# --- Step 1: Initialize Keywords ---
event_keywords = {"birthday", "wedding", "party", "meeting"}
valid_months = {
    "january", "february", "march", "april", "may", "june",
    "july", "august", "september", "october", "november", "december"
}
known_locations = {
    "chuck e. cheese", "dave & busters", "home", "garage", "downtown",
    "new york", "chicago", "san francisco"
}


# --- Step 2: Define Prompts Based on Missing Info ---
def next_question(memory):
    if memory["event_type"] is None:
        return "What type of event are you planning? (e.g. birthday, wedding)"
    elif memory["date"] is None:
        return "When is the event?"
    elif memory["location"] is None:
        return "Where will it be held?"
    elif memory["guests"] is None:
        return "How many guests are you expecting?"
    else:
        return None

# --- Step 3: 🔧 Updated update_memory() Function: ---
def update_memory(user_input, memory):
    user_input = user_input.lower()
    response = None  # ← Track what to say back

    def extract_keyword(user_input, keywords):
        for word in keywords:
            if word in user_input:
                return word
        return None

    # --- Event type ---
    if memory["event_type"] is None:
        event = extract_keyword(user_input, event_keywords)
        if event:
            memory["event_type"] = event
            response = f"Great! You're planning a {event}."

    # --- Date ---
    elif memory["date"] is None:
        month = extract_keyword(user_input, valid_months)
        if month:
            memory["date"] = month
            response = f"Noted — it's in {month.title()}."

    # --- Location ---
    elif memory["location"] is None:
        location = extract_keyword(user_input, known_locations)
        if location:
            memory["location"] = location
            response = f"Got it! It'll be held at {location.title()}."

    # --- Guests ---
    elif memory["guests"] is None:
        digits = "".join(char for char in user_input if char.isdigit())
        if digits:
            memory["guests"] = digits
            response = f"Perfect — expecting {digits} guests."

    return memory, response


def extract_keyword(user_input, keyword_set):
    user_input = user_input.lower()
    for word in keyword_set:
        if word in user_input:
            return word
    return None

# --- Step 4: Interaction Loop ---
def run_event_planner():
    # 🧠 Initialize memory
    memory = {
        "event_type": None,
        "date": None,
        "location": None,
        "guests": None
    }

    print("🧳 Event Planning Agent is ready!")

    while True:
        question = next_question(memory)
        if question:
            print("🤖 Agent:", question)
            user_input = input("👤 You: ")
            memory, confirmation = update_memory(user_input, memory)
            if confirmation:
                print("🤖 Agent:", confirmation)
        else:
            # Summary & confirmation loop
            while True:
                print("\n✅ Event Summary:")
                for k, v in memory.items():
                    print(f"{k}: {v}")

                confirm = input("\n🤖 Agent: Would you like to update anything? (yes/no)\n👤 You: ").strip().lower()

                if confirm == "no":
                    print("\n🎉 Final Event Plan Confirmed:")
                    for k, v in memory.items():
                        print(f"{k}: {v}")
                    print("👋 Thank you! Your event has been recorded. Goodbye!\n")
                    return memory  # ✅ Exit entire function

                elif confirm == "yes":
                    field = input("🤖 Agent: What would you like to change? (event_type, date, location, guests)\n👤 You: ").strip().lower()
                    if field in memory:
                        new_val = input(f"🤖 Agent: What should I update '{field}' to?\n👤 You: ").strip()
                        memory[field] = new_val
                        print(f"🤖 Agent: Got it! Updated {field} to {new_val}.")
                    else:
                        print("🤖 Agent: I didn't recognize that field. Try again.")
                else:
                    print("🤖 Agent: Please respond with 'yes' or 'no'.")


# Run it
run_event_planner()

🧳 Event Planning Agent is ready!
🤖 Agent: What type of event are you planning? (e.g. birthday, wedding)
👤 You: Bah Mitzvah
🤖 Agent: What type of event are you planning? (e.g. birthday, wedding)
👤 You: Wedding
🤖 Agent: Great! You're planning a wedding.
🤖 Agent: When is the event?
👤 You: July
🤖 Agent: Noted — it's in July.
🤖 Agent: Where will it be held?
👤 You: Tokyo
🤖 Agent: Where will it be held?
👤 You: Chicagp
🤖 Agent: Where will it be held?
👤 You: Chicagp
🤖 Agent: Where will it be held?
👤 You: Chicago
🤖 Agent: Got it! It'll be held at Chicago.
🤖 Agent: How many guests are you expecting?
👤 You: 1
🤖 Agent: Perfect — expecting 1 guests.

✅ Event Summary:
event_type: wedding
date: july
location: chicago
guests: 1

🤖 Agent: Would you like to update anything? (yes/no)
👤 You: no

🎉 Final Event Plan Confirmed:
event_type: wedding
date: july
location: chicago
guests: 1
👋 Thank you! Your event has been recorded. Goodbye!



{'event_type': 'wedding', 'date': 'july', 'location': 'chicago', 'guests': '1'}

### 🔮 What We’ll Add Next

We’ll insert the LLM at the **input understanding** step:

```
User input → 🔍 LLM clarifies → 🧠 Store clean value in memory
```

---

### 💡 Here’s What We’ll Do in the Next Step:

1. **Load a small model** like `flan-t5-base` or `flan-t5-large` (depending on Colab performance)
2. **Create a helper function** like `llm_extract_slot(user_input, slot_type)`
3. Use it for slots like:
   - Event type
   - Date (month)
   - Location
   - Guest count

---

### 🛠️ Example

```python
# Example use
slot = "event_type"
user_input = "actually it’s more of a wedding than a birthday"
clean_value = extract_slot_with_llm(user_input, slot)

# returns: "wedding"
```



In [3]:
# 📦 Step 1: Install & Import
# !pip install -q transformers

from transformers import pipeline
import re, json

# Load a lightweight instruction-tuned model
generator = pipeline("text2text-generation", model="google/flan-t5-base")


Device set to use cpu


## High Quality Prompt

For AI agents, the **prompt is the operating manual** — and if you’re aiming for high-quality, predictable behavior from your LLM, a rich, well-structured prompt with explicit rules and examples is one of the **best investments** you can make.

Let’s build a strong, few-shot prompt together.

---

### 🧠 GOAL:
Create a **few-shot prompt** that guides the LLM to extract:
- `event_type`: e.g. "birthday", "wedding", "party"
- `date`: a month name
- `location`: free text or city
- `guests`: a number


### ✅ Why This Works

- It shows multiple **realistic variations**
- Includes **rules and formatting expectations**
- Teaches the model what to do for **partial inputs**
- Makes casing and fallback behavior predictable

#Multi-Turn AI Event Planning Agent

In [16]:
from difflib import get_close_matches

def fuzzy_match(word, options):
    matches = get_close_matches(word.lower(), [opt.lower() for opt in options], n=1, cutoff=0.6)
    return matches[0] if matches else None


# Define the reusable prompt (with a placeholder)
llm_prompt_template = """
You are an intelligent event planning assistant.

Your job is to extract structured information from user messages and return it as valid JSON.

Rules:
- Only respond with a JSON object using the keys: "event_type", "date", "location", "guests"
- If any value is unknown or missing, set it to null
- Do NOT explain or add extra text
- Always return lowercase values (except city names)

Examples:
Message: "We’re hosting a birthday party in June for about 10 kids."
Response: { "event_type": "birthday", "date": "june", "location": null, "guests": "10" }

Message: "Throwing a wedding at the beach in July"
Response: { "event_type": "wedding", "date": "july", "location": "the beach", "guests": null }

Message: "Our event is in New York and we’re expecting 50 people"
Response: { "event_type": null, "date": null, "location": "New York", "guests": "50" }

Message: "It's a corporate conference happening in October"
Response: { "event_type": "conference", "date": "october", "location": null, "guests": null }

Message: "Planning an anniversary dinner for 2 in Paris in May"
Response: { "event_type": "anniversary", "date": "may", "location": "Paris", "guests": "2" }

Now extract the fields for this message:
Message: "{user_input}"
Response:
""".strip()

# --- Step 1: Initialize Keywords ---
event_keywords = {"birthday", "wedding", "party", "meeting"}

valid_months = {
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
}

known_locations = {
    "Chuck E. Cheese", "Dave & Busters", "Home", "Garage", "Downtown",
    "New York", "Chicago", "San Francisco"
}


# --- Step 2: Define Prompts Based on Missing Info ---
def next_question(memory):
    if memory["event_type"] is None:
        return "What type of event are you planning? (e.g. birthday, wedding)"
    elif memory["date"] is None:
        return "When is the event?"
    elif memory["location"] is None:
        return "Where will it be held?"
    elif memory["guests"] is None:
        return "How many guests are you expecting?"
    else:
        return None

# --- Step 3: 🔧 Updated update_memory() Function: ---
def update_memory(user_input, memory):
    response = None
    updated = False

    # Try quick keyword matches first
    event = extract_keyword(user_input, event_keywords)
    month = extract_keyword(user_input, valid_months)
    location = extract_keyword(user_input, known_locations)
    digits = "".join(char for char in user_input if char.isdigit())

    # If any are missing, call the LLM
    if not event or not month or not location or not digits:
        llm_result = fallback_extract(user_input)
        try:
            fields = json.loads(llm_result)
            if not isinstance(fields, dict):
                fields = {}  # ✅ Defensive fallback
        except:
            fields = {}

        event = event or fields.get("event_type")
        month = month or fields.get("date")
        location = location or fields.get("location")
        digits = digits or fields.get("guests")

    # Update memory + build response
    if memory["event_type"] is None and event:
        memory["event_type"] = event
        response = f"Great! You're planning a {event.title()}."
    elif memory["date"] is None and month:
        memory["date"] = month
        response = f"Noted — it's in {month.title()}."
    elif memory["location"] is None and location:
        memory["location"] = location
        response = f"Got it! It'll be held at {location.title()}."
    elif memory["guests"] is None and digits:
        memory["guests"] = digits
        response = f"Perfect — expecting {digits} guests."

    # Inside the confirmation block, after confirm == "no"
    summary_prompt = f"""
    Create a cheerful confirmation for this event info:

    event_type: {memory['event_type']}
    date: {memory['date']}
    location: {memory['location']}
    guests: {memory['guests']}

    Use a friendly tone.
    """.strip()

    return memory, response


# --- Direct keyword-based extraction ---
# def extract_keyword(user_input, keyword_set):
#     user_input = user_input.lower()
#     for keyword in keyword_set:
#         if keyword.lower() in user_input:
#             return keyword  # return it with proper casing
#     return None

# ✅ Use fuzzy_match() for cities and months
def extract_keyword(user_input, keyword_set):
    return fuzzy_match(user_input, keyword_set)


# Fallback with LLM
def fallback_extract(user_input):
    prompt = llm_prompt_template.replace("{user_input}", user_input)
    return generator(prompt, max_new_tokens=30)[0]["generated_text"].strip()


# --- Step 4: Interaction Loop ---
def run_event_planner():
    # 🧠 Initialize memory
    memory = {
        "event_type": None,
        "date": None,
        "location": None,
        "guests": None
    }

    print("🧳 Event Planning Agent is ready!")

    while True:
        # Identify which field is still missing
        missing_field = next((k for k, v in memory.items() if v is None), None)
        question = clarify_field(missing_field) if missing_field else None

        if question:
            print("🤖 Agent:", question)
            user_input = input("👤 You: ")
            memory, confirmation = update_memory(user_input, memory)
            if confirmation:
                print("🤖 Agent:", confirmation)
        else:
            # 🎯 All fields are filled, enter confirmation loop
            while True:
                print("\n✅ Event Summary:")
                for k, v in memory.items():
                    print(f"{k}: {v}")

                confirm = input("\n🤖 Agent: Would you like to update anything? (yes/no)\n👤 You: ").strip().lower()

                if confirm == "no":
                    # ✅ Final LLM-generated message
                    summary_prompt = f"""
                    You are an event planning assistant. Generate a friendly confirmation message for the user based on the following details:

                    Event: {memory['event_type']}
                    Date: {memory['date']}
                    Location: {memory['location']}
                    Guests: {memory['guests']}

                    Your message should be short, positive, and clearly confirm all details.
                    """.strip()

                    summary = generator(summary_prompt, max_new_tokens=60)[0]["generated_text"]
                    print("\n🎉 Final Event Plan Confirmed:")
                    for k, v in memory.items():
                        print(f"{k}: {v}")
                    print("🤖 Agent:", summary)
                    return memory

                elif confirm == "yes":
                    field = input("🤖 Agent: What would you like to change? (event_type, date, location, guests)\n👤 You: ").strip().lower()
                    if field in memory:
                        new_val = input(f"🤖 Agent: What should I update '{field}' to?\n👤 You: ").strip()
                        memory[field] = new_val
                        print(f"🤖 Agent: Got it! Updated {field} to {new_val}.")
                    else:
                        print("🤖 Agent: I didn't recognize that field. Try again.")
                else:
                    print("🤖 Agent: Please respond with 'yes' or 'no'.")


def clarify_field(field):
    questions = {
        "event_type": "What type of event are you planning? (e.g. birthday, wedding)",
        "date": "When is the event?",
        "location": "Where will it be held?",
        "guests": "How many guests are you expecting?"
    }
    return questions.get(field, "Can you clarify?")


# Run it
run_event_planner()

🧳 Event Planning Agent is ready!
🤖 Agent: What type of event are you planning? (e.g. birthday, wedding)
👤 You: I am thinking about a Wedding
🤖 Agent: What type of event are you planning? (e.g. birthday, wedding)
👤 You: wedding
🤖 Agent: Great! You're planning a Wedding.
🤖 Agent: When is the event?
👤 You: this summer
🤖 Agent: When is the event?
👤 You: july
🤖 Agent: Noted — it's in July.
🤖 Agent: Where will it be held?
👤 You: Miami
🤖 Agent: Where will it be held?
👤 You: miami
🤖 Agent: Where will it be held?
👤 You: Chicago
🤖 Agent: Got it! It'll be held at Chicago.
🤖 Agent: How many guests are you expecting?
👤 You: 10
🤖 Agent: Perfect — expecting 10 guests.

✅ Event Summary:
event_type: wedding
date: july
location: chicago
guests: 10

🤖 Agent: Would you like to update anything? (yes/no)
👤 You: no

🎉 Final Event Plan Confirmed:
event_type: wedding
date: july
location: chicago
guests: 10
🤖 Agent: The event is a wedding in Chicago. The date is July. The venue is chicago. The guests are 10 peo

{'event_type': 'wedding',
 'date': 'july',
 'location': 'chicago',
 'guests': '10'}

In [19]:
from difflib import get_close_matches

def fuzzy_match(word, options):
    matches = get_close_matches(word.lower(), [opt.lower() for opt in options], n=1, cutoff=0.6)
    return matches[0] if matches else None


# Define the reusable prompt (with a placeholder)
llm_prompt_template = """
You are an intelligent event planning assistant.

Your job is to extract structured information from user messages and return it as valid JSON.

Rules:
- Only respond with a JSON object using the keys: "event_type", "date", "location", "guests"
- If any value is unknown or missing, set it to null
- Do NOT explain or add extra text
- Always return lowercase values (except city names)

Examples:
Message: "We’re hosting a birthday party in June for about 10 kids."
Response: { "event_type": "birthday", "date": "june", "location": null, "guests": "10" }

Message: "Throwing a wedding at the beach in July"
Response: { "event_type": "wedding", "date": "july", "location": "the beach", "guests": null }

Message: "Our event is in New York and we’re expecting 50 people"
Response: { "event_type": null, "date": null, "location": "New York", "guests": "50" }

Message: "It's a corporate conference happening in October"
Response: { "event_type": "conference", "date": "october", "location": null, "guests": null }

Message: "Planning an anniversary dinner for 2 in Paris in May"
Response: { "event_type": "anniversary", "date": "may", "location": "Paris", "guests": "2" }

Now extract the fields for this message:
Message: "{user_input}"
Response:
""".strip()

# --- Step 1: Updated Prompt for LLM Extraction ---

fallback_prompt_template = """
You are an event planning assistant. Extract the structured information from the user message.

Instructions:
- Extract four fields: "event_type", "date", "location", and "guests"
- If a field is not mentioned, return null for it.
- Return only a valid JSON object, nothing else.

Examples:

Message: "I'd like to plan a wedding in July"
Response: {{ "event_type": "wedding", "date": "july", "location": null, "guests": null }}

Message: "I'm hosting a party in San Francisco this August, about 20 guests"
Response: {{ "event_type": "party", "date": "august", "location": "san francisco", "guests": "20" }}

Now extract info from this message:
Message: "{message}"
Response:
""".strip()


# --- Step 1: Initialize Keywords ---
event_keywords = {"birthday", "wedding", "party", "meeting"}

valid_months = {m.lower() for m in [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]}

known_locations = {c.lower() for c in [
    "Chuck E. Cheese", "Dave & Busters", "Miami", "Atlanta", "Houston",
    "New York", "Chicago", "San Francisco"
]}


# --- Step 2: Define Prompts Based on Missing Info ---
def next_question(memory):
    if memory["event_type"] is None:
        return "What type of event are you planning? (e.g. birthday, wedding)"
    elif memory["date"] is None:
        return "When is the event?"
    elif memory["location"] is None:
        return "Where will it be held?"
    elif memory["guests"] is None:
        return "How many guests are you expecting?"
    else:
        return None

# --- Step 3: 🔧 Updated update_memory() Function: ---
def update_memory(user_input, memory):
    response = None
    updated = False

    # Try quick keyword matches first
    event = extract_keyword(user_input, event_keywords)
    month = extract_keyword(user_input, valid_months)
    location = extract_keyword(user_input, known_locations)
    digits = "".join(char for char in user_input if char.isdigit())

    # If any are missing, call the LLM
    if not event or not month or not location or not digits:
        llm_result = fallback_extract(user_input)
        try:
            fields = json.loads(llm_result)
            if not isinstance(fields, dict):
                fields = {}  # ✅ Defensive fallback
        except:
            fields = {}

        event = event or fields.get("event_type")
        month = month or fields.get("date")
        location = location or fields.get("location")
        digits = digits or fields.get("guests")

    # Update memory + build response
    if memory["event_type"] is None and event:
        memory["event_type"] = event
        response = f"Great! You're planning a {event.title()}."
    elif memory["date"] is None and month:
        memory["date"] = month
        response = f"Noted — it's in {month.title()}."
    elif memory["location"] is None and location:
        memory["location"] = location
        response = f"Got it! It'll be held at {location.title()}."
    elif memory["guests"] is None and digits:
        memory["guests"] = digits
        response = f"Perfect — expecting {digits} guests."

    # Inside the confirmation block, after confirm == "no"
    summary_prompt = f"""
    Create a cheerful confirmation for this event info:

    event_type: {memory['event_type']}
    date: {memory['date']}
    location: {memory['location']}
    guests: {memory['guests']}

    Use a friendly tone.
    """.strip()

    return memory, response

# ✅ Use fuzzy_match() for cities and months
def extract_keyword(user_input, keyword_set):
    return fuzzy_match(user_input, keyword_set)

# --- Step 2: Function to Use Prompt with LLM ---
def fallback_extract(user_input):
    prompt = fallback_prompt_template.format(message=user_input)
    result = generator(prompt, max_new_tokens=100)[0]["generated_text"].strip()
    return result


# --- Step 4: Interaction Loop ---
def run_event_planner():
    # 🧠 Initialize memory
    memory = {
        "event_type": None,
        "date": None,
        "location": None,
        "guests": None
    }

    print("🧳 Event Planning Agent is ready!")

    while True:
        # Identify which field is still missing
        missing_field = next((k for k, v in memory.items() if v is None), None)
        question = clarify_field(missing_field) if missing_field else None

        if question:
            print("🤖 Agent:", question)
            user_input = input("👤 You: ")
            memory, confirmation = update_memory(user_input, memory)
            if confirmation:
                print("🤖 Agent:", confirmation)
        else:
            # 🎯 All fields are filled, enter confirmation loop
            while True:
                print("\n✅ Event Summary:")
                for k, v in memory.items():
                    print(f"{k}: {v}")

                confirm = input("\n🤖 Agent: Would you like to update anything? (yes/no)\n👤 You: ").strip().lower()

                if confirm == "no":
                    # ✅ Final LLM-generated message
                    summary_prompt = f"""
                    You are an event planning assistant. Generate a friendly confirmation message for the user based on the following details:

                    Event: {memory['event_type']}
                    Date: {memory['date']}
                    Location: {memory['location']}
                    Guests: {memory['guests']}

                    Your message should be short, positive, and clearly confirm all details.
                    """.strip()

                    summary = generator(summary_prompt, max_new_tokens=60)[0]["generated_text"]
                    print("\n🎉 Final Event Plan Confirmed:")
                    for k, v in memory.items():
                        print(f"{k}: {v}")
                    print("🤖 Agent:", summary)
                    return memory

                elif confirm == "yes":
                    field = input("🤖 Agent: What would you like to change? (event_type, date, location, guests)\n👤 You: ").strip().lower()
                    if field in memory:
                        new_val = input(f"🤖 Agent: What should I update '{field}' to?\n👤 You: ").strip()
                        memory[field] = new_val
                        print(f"🤖 Agent: Got it! Updated {field} to {new_val}.")
                    else:
                        print("🤖 Agent: I didn't recognize that field. Try again.")
                else:
                    print("🤖 Agent: Please respond with 'yes' or 'no'.")


def clarify_field(field):
    questions = {
        "event_type": "What type of event are you planning? (e.g. birthday, wedding)",
        "date": "When is the event?",
        "location": "Where will it be held?",
        "guests": "How many guests are you expecting?"
    }
    return questions.get(field, "Can you clarify?")


# Run it
run_event_planner()

🧳 Event Planning Agent is ready!
🤖 Agent: What type of event are you planning? (e.g. birthday, wedding)
👤 You: a wedding
🤖 Agent: Great! You're planning a Wedding.
🤖 Agent: When is the event?
👤 You: this summer
🤖 Agent: When is the event?
👤 You: in July
🤖 Agent: Noted — it's in July.
🤖 Agent: Where will it be held?
👤 You: in the south
🤖 Agent: Where will it be held?
👤 You: Miami
🤖 Agent: Got it! It'll be held at Miami.
🤖 Agent: How many guests are you expecting?
👤 You: 10
🤖 Agent: Perfect — expecting 10 guests.

✅ Event Summary:
event_type: wedding
date: july
location: miami
guests: 10

🤖 Agent: Would you like to update anything? (yes/no)
👤 You: yes
🤖 Agent: What would you like to change? (event_type, date, location, guests)
👤 You: event_type
🤖 Agent: What should I update 'event_type' to?
👤 You: party
🤖 Agent: Got it! Updated event_type to party.

✅ Event Summary:
event_type: party
date: july
location: miami
guests: 10

🤖 Agent: Would you like to update anything? (yes/no)
👤 You: no

🎉 

{'event_type': 'party', 'date': 'july', 'location': 'miami', 'guests': '10'}

### ✅ What’s Working Well

| Feature | Behavior |
|--------|----------|
| **LLM + fallback** | Works smoothly when the user gives vague input like “this summer” and “in the south.” |
| **Memory filling** | Collects and tracks all required fields across turns. |
| **Confirmation loop** | Lets the user update specific parts of the event cleanly and clearly. |
| **Friendly responses** | The assistant gives natural feedback like “Got it!” and “Perfect — expecting 10 guests.” |
| **Final summary** | Clear output, both as chat text and structured `dict`. |

---

### 📘 You’ve Learned Core Agent Skills

What you’ve implemented so far covers *key components of real-world AI agents*:

| Skill | Purpose |
|-------|---------|
| 🧠 **Stateful memory** | Track what’s been filled and what’s still missing |
| 🎯 **Goal-directed questioning** | Guide the user through completing a task |
| 🪄 **LLM fallback extraction** | Handle fuzzy input when keyword matching fails |
| 🔄 **Clarification and confirmation** | Let users revise answers dynamically |
| 🧹 **Output post-processing** | Clean up and validate model responses |

---

### 🛠️ Next Steps (Optional Ideas)

If you want to keep learning and flex more muscles:

- **Improve LLM summary:** Use a better prompt to generate a final natural-sounding confirmation message (“You’re planning a party in Miami this July with 10 guests.”).
- **Log memory for analysis:** Store conversation logs or structured outputs to a `.jsonl` or database.
- **Add fuzzy logic validation:** Handle things like “20 to 30 people” → estimate or ask clarification.
- **Support synonyms:** Like "nuptials" or "get-together" for event types.



